CPython 3.7.6
IPython 7.12.0

numpy 1.18.1
pandas 1.0.1
torch 1.4.0

In [1]:
import torch

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters
from torch import nn, optim

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 14, 10
register_matplotlib_converters()

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

## Check cuda is available:
Output will return True is cuda is available and False is not.

In [2]:
torch.cuda.is_available()

False

## Package Version Check
Install Watermark to check package versions.

Versions used here are-

    CPython 3.7.6
    IPyhon 7.12.0

    numpy: 1.18.1
    pandas: 1.0.1
    torch: 1.4.0

In [3]:
!pip install -Uq watermark
%reload_ext watermark
%watermark -v -p numpy,pandas,torch

CPython 3.7.6
IPython 7.12.0

numpy 1.18.1
pandas 1.0.1
torch 1.4.0


## Import Data:
Read in the robot joint and force data.
Here, the data was clipped to 5000 because the local computer/GPU would run out of memery if any higher.
This was before it was descovered that batches of data could be fetched individually. 

In [4]:
data = pd.read_csv("/home/ur10pc/Desktop/robot_data2/80k_data/all_joints_and_force.csv")
data = data[:5000]
data.head()

,Force Vec,joint_0,joint_1,joint_2,joint_3,joint_4,joint_5
0,0.030003,1.404830,-1.171765,2.483047,-2.862827,-1.566874,2.720634
1,0.185473,1.404794,-1.171765,2.483070,-2.862839,-1.566886,2.720623
2,0.090554,1.404842,-1.171752,2.483034,-2.862851,-1.566910,2.720611
3,0.100997,1.404818,-1.171752,2.483070,-2.862839,-1.566910,2.720634
4,0.054774,1.404806,-1.171777,2.483058,-2.862863,-1.566886,2.720611


## Smooth Force Data:
Using the pandas mean method with a rolling window of 200 samples. The number of samples was found experimentally and provided good noise reduction while maintaining a good trend. 

In [5]:
print(data.iloc[:,0].head())
data['force_mean'] = data.iloc[:,0].rolling(window=200).mean()
data.head(10)

0    0.030003
1    0.185473
2    0.090554
3    0.100997
4    0.054774
Name: Force Vec, dtype: float64


,Force Vec,joint_0,joint_1,joint_2,joint_3,joint_4,joint_5,force_mean
0,0.030003,1.404830,-1.171765,2.483047,-2.862827,-1.566874,2.720634,NaN
1,0.185473,1.404794,-1.171765,2.483070,-2.862839,-1.566886,2.720623,NaN
2,0.090554,1.404842,-1.171752,2.483034,-2.862851,-1.566910,2.720611,NaN
3,0.100997,1.404818,-1.171752,2.483070,-2.862839,-1.566910,2.720634,NaN
4,0.054774,1.404806,-1.171777,2.483058,-2.862863,-1.566886,2.720611,NaN
5,0.325884,1.404782,-1.171777,2.483034,-2.862851,-1.566910,2.720623,NaN
6,0.273862,1.404818,-1.171777,2.483082,-2.862863,-1.566898,2.720611,NaN
7,0.073489,1.404842,-1.171765,2.483058,-2.862851,-1.566886,2.720611,NaN
8,0.083067,1.404818,-1.171765,2.483070,-2.862815,-1.566886,2.720623,NaN
9,0.054776,1.404830,-1.171789,2.483047,-2.862851,-1.566922,2.720646,NaN


## Format Data
As the first 200 samples from the first trajectory have been used to smooth the proceeding samples, the trajectory itself can no longer be used, therefore the entire trajectory needs to be removed. 
Plus, we no longer need the 'Force Vec' column so that can be dropped.

In [6]:
data = data[1000:5000].reset_index(drop=True)
data = data.drop(['Force Vec'], axis=1)
data.shape
data.head()

,joint_0,joint_1,joint_2,joint_3,joint_4,joint_5,force_mean
0,1.404734,-1.171789,2.483022,-2.862731,-1.566982,2.720730,6.751787
1,1.404770,-1.171825,2.483047,-2.862755,-1.567006,2.720706,6.728010
2,1.404722,-1.171849,2.483022,-2.862755,-1.567018,2.720730,6.698278
3,1.404746,-1.171800,2.483034,-2.862755,-1.567006,2.720730,6.683689
4,1.404746,-1.171812,2.483058,-2.862743,-1.566994,2.720730,6.669569


## Check DataFrame Shape
It's always wize to check the shape of your dataframes.

In [7]:
data.shape

(4000, 7)

### Scale data

In [8]:
features_scaler = data
target_scaler = np.asarray(data['force_mean'])

scaler = MinMaxScaler(feature_range=(-1, 1))

scaler1 = scaler.fit(features_scaler)
features = scaler1.transform(features_scaler)

scaler2 = scaler.fit(target_scaler.reshape(-1, 1))
targets = scaler2.transform(target_scaler.reshape(-1, 1))


In [9]:
test_data_size = 500
X_train = features[:-test_data_size]
X_test = features[-test_data_size:]
y_train = targets[:-test_data_size]
y_test = targets[-test_data_size:]


In [10]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(3500, 7) (3500, 1) (500, 7) (500, 1)


In [11]:
X_test.shape

(500, 7)

In [12]:
# features_scaler = data
# target_scaler = np.asarray(data['force_mean'])

# scaler = MinMaxScaler(feature_range=(-1, 1))

# scaler1 = scaler.fit(features_scaler)
# X_train = scaler1.transform(X_train)



# scaler2 = scaler.fit(target_scaler.reshape(-1,1))
# y_train = scaler2.transform(y_train.reshape(-1,1))

# scaler3 = scaler.fit(features_scaler)
# X_test = scaler3.transform(X_test)


# scaler4 = scaler.fit(target_scaler.reshape(-1,1))
# y_test = scaler4.transform(y_test)


# train_data_normalized = scaler.fit_transform(train_data .reshape(-1, 1))
# test_data_normalized = scaler.fit_transform(test_data .reshape(-1, 1))


# scaler = MinMaxScaler(feature_range=(-1, 1))

# scaler = scaler.fit(train_data)

# train_data = scaler.transform(train_data)

# test_data = scaler.transform(test_data)

# train_data_normalized = scaler.fit_transform(train_data .reshape(-1, 1))
# test_data_normalized = scaler.fit_transform(test_data .reshape(-1, 1))


In [13]:

print(X_train[:4],y_train[:4])

[[-0.80722161 -0.85257162  0.82227675 -0.79638504  0.22971698  0.82490527
   0.22603083]
 [-0.80714234 -0.85265416  0.8223018  -0.79643066  0.22370283  0.82489132
   0.22051132]
 [-0.80724821 -0.85270846  0.82227675 -0.79643066  0.22075472  0.82490527
   0.21360962]
 [-0.80719527 -0.85259877  0.82228927 -0.79643066  0.22370283  0.82490527
   0.21022298]] [[0.22603083]
 [0.22051132]
 [0.21360962]
 [0.21022298]]


In [14]:

def train_create_sequences( seq_length):
    xs = []
    ys = []

    for i in range(len(X_train)-seq_length-seq_length):
        x = X_train[i:(i+seq_length)]
        #print(i,(i+seq_length))
        y = y_train[i+seq_length]
        #print(i+seq_length+1)
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

def test_create_sequences(seq_length):
    xs = []
    ys = []

    for i in range(len(X_test)-seq_length-seq_length):
        x = X_test[i:(i+seq_length)]
        #print(i,(i+seq_length))
        y = y_test[i+seq_length]
        #print(i+seq_length+1)
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

In [15]:
X_train[:4]

array([[-0.80722161, -0.85257162,  0.82227675, -0.79638504,  0.22971698,
         0.82490527,  0.22603083],
       [-0.80714234, -0.85265416,  0.8223018 , -0.79643066,  0.22370283,
         0.82489132,  0.22051132],
       [-0.80724821, -0.85270846,  0.82227675, -0.79643066,  0.22075472,
         0.82490527,  0.21360962],
       [-0.80719527, -0.85259877,  0.82228927, -0.79643066,  0.22370283,
         0.82490527,  0.21022298]])

## Convert Sequences to CUDA Tensors
Here we call the train_create_sequences and test_create_sequences functions to build our sequence data and then convert them into torch.cuda tensors for use on the GPU.
If you are not using a GPU, then you would instead use torch.FloatTensor(). This conversion is necessery for Torch to manipulate tensors using built in attributes and for casting to a GPU if necessary. 

In [16]:
seq_length = 5 ## Given 5 samples, what will the force be on the 6th?

X_train, y_train = train_create_sequences(seq_length)
X_test, y_test = test_create_sequences(seq_length)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

X_train = torch.cuda.FloatTensor(X_train)
y_train = torch.cuda.FloatTensor(y_train)
X_test = torch.cuda.FloatTensor(X_test)
y_test = torch.cuda.FloatTensor(y_test)



(3490, 5, 7) (3490, 1)
(490, 5, 7) (490, 1)


RuntimeError: cuda runtime error (804) : forward compatibility was attempted on non supported HW at /opt/conda/conda-bld/pytorch_1579022060824/work/aten/src/THC/THCGeneral.cpp:50

In [ ]:
class ForcePredictor(nn.Module):

  def __init__(self, n_features, n_hidden, seq_len, n_layers=2):
    super(ForcePredictor, self).__init__()

    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        print("Running on the GPU")
    else:
        device = torch.device("cpu")
        print("Running on CPU")
    
    self.n_hidden = n_hidden
    self.seq_len = seq_len
    self.n_layers = n_layers

    self.lstm = nn.LSTM(
      input_size=n_features,
      hidden_size=n_hidden,
      num_layers=n_layers,
      dropout=0.2
    )

    self.linear = nn.Linear(in_features=n_hidden, out_features=1)
    
    
    
  def reset_hidden_state(self):
    self.hidden = (
        torch.zeros(self.n_layers, self.seq_len, self.n_hidden).to(device),
        torch.zeros(self.n_layers, self.seq_len, self.n_hidden).to(device)
    )

  def forward(self, sequences):
    lstm_out, self.hidden = self.lstm(
      sequences.view(len(sequences), self.seq_len, -1),
      self.hidden
    )
    last_time_step = \
      lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]
    y_pred = self.linear(last_time_step)
    return y_pred

In [ ]:
def train_model(
  model, 
  train_data, 
  train_labels, 
  test_data=None, 
  test_labels=None
):
  model = model.to(device)

  loss_fn = torch.nn.MSELoss(reduction='mean')

  optimiser = torch.optim.Adam(model.parameters(), lr=0.00001)
  num_epochs =1000

  train_hist = np.zeros(num_epochs)
  test_hist = np.zeros(num_epochs)

  for t in range(num_epochs):
    model.reset_hidden_state()

    y_pred = model(X_train)

    loss = loss_fn(y_pred.float(), y_train)

    if test_data is not None:
      with torch.no_grad():
        y_test_pred = model(X_test)
        test_loss = loss_fn(y_test_pred.float(), y_test)
      test_hist[t] = test_loss.item()

      if t % 10 == 0:  
        print(f'Epoch {t} train loss: {loss.item()} test loss: {test_loss.item()}')
    elif t % 10 == 0:
      print(f'Epoch {t} train loss: {loss.item()}')

    train_hist[t] = loss.item()
    
    optimiser.zero_grad()

    loss.backward()

    optimiser.step()
  
  return model.eval(), train_hist, test_hist

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")
    

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
model = ForcePredictor(
  n_features=7, 
  n_hidden=512, 
  seq_len=seq_length, 
  n_layers=2
)

model, train_hist, test_hist = train_model(
  model, 
  X_train, 
  y_train, 
  X_test, 
  y_test
)

In [ ]:
plt.plot(train_hist, label="Training loss")
plt.plot(test_hist, label="Test loss")
#plt.ylim((0, 5))
plt.legend();

In [ ]:
with torch.no_grad():
  test_seq = X_test[:1]
  preds = []
  for i in range(len(X_test)):
    y_test_pred = model(test_seq).to(device)
    pred = torch.flatten(y_test_pred).item()
    preds.append(pred)
    new_seq = X_test[i] 
    #print(X_test[i])
    test_seq = torch.cuda.FloatTensor(new_seq).view(1, seq_length, -1)
   

In [ ]:

true_cases = scaler.inverse_transform(y_test.cpu())#.flatten().numpy())
#print(len(y_test))
#print(len(true_cases))

predicted_cases = scaler.inverse_transform(
  np.expand_dims(preds, axis=0)
).flatten()
#print(predicted_cases)

## Notes to Self:
Need to remove the last bit of data to make predictions on!!!
or just overlay the data.
Need to use all data- shuffle-normalise. 
Then split into training, validation and testing.
Then feed in 1000 samples at a time. If 1000 samples is one epoch, the network will reset  the hidden state after each 1000 samples = 1 trajectory. 

## Plot Predictions
As this is only the beginning of training, testing is done on the training data to see how accurate the model is at making predictions on data that it has already seen. Only once the model can accuratly predict forces on known data can we start to look at generalising to unseen data. 
As you can see from the chart below, the model is not yet ready to start making predictions on unseen data.

In [ ]:
daily_cases = X_train[:,-1]
daily_cases = pd.DataFrame(X_train)
y_train = y_train.cpu()

plt.plot(true_cases,label='Real Force')
plt.plot(predicted_cases,label='Predicted Force')

plt.legend();